In [14]:
import json
import pandas as pd
import os
from tqdm import tqdm
from pydub import AudioSegment  
import matplotlib.pyplot as plt
root_dirs = {
    "Birdsong" : "/nlsasfs/home/nltm-pilot/sandeshk/icassp/data/birdsong",
    "IEMOCAP"  : "/speech/Databases/Birdsong/IEMOCAP/",
    "MusicalInstruments" : "/speech/Databases/Birdsong/MusicalInstruments",
    "speechcommandsv1" : "/nlsasfs/home/nltm-pilot/sandeshk/icassp/data/SpeechCommandsV1/train"
}

# BirdSong
```shell
python data_prep.py --root_dir "/nlsasfs/home/nltm-pilot/sandeshk/icassp/data/birdsong/freefield1010" --no_workers 10 --prefix wav --suffix spec
```

In [11]:
root_dir = root_dirs["Birdsong"]

ff_txt = os.path.join(root_dir,"freefield1010/6035814")
df1 = pd.read_csv(ff_txt) 
df1['Path'] = df1.apply(lambda row: os.path.join( 'freefield1010','spec',str(row.itemid)+'.wav.npy'), axis = 1)
df1['PathL2'] = df1.apply(lambda row: os.path.join( 'freefield1010','specl2',str(row.itemid)+'.wav.npy'), axis = 1)
df1['AudioPath'] = df1.apply(lambda row: os.path.join( 'freefield1010','wav',str(row.itemid)+'.wav'), axis = 1)
df1.rename(columns = {'hasbird':'Label'}, inplace = True)
wab_txt = os.path.join(root_dir,"Warblr/6035817")
df2 = pd.read_csv(wab_txt) 
df2['Path'] = df2.apply(lambda row: os.path.join( 'Warblr','spec',str(row.itemid)+'.wav.npy'), axis = 1)
df2['PathL2'] = df2.apply(lambda row: os.path.join( 'Warblr','specl2',str(row.itemid)+'.wav.npy'), axis = 1)
df2['AudioPath'] = df2.apply(lambda row: os.path.join( 'Warblr','wav',str(row.itemid)+'.wav'), axis = 1)
df2.rename(columns = {'hasbird':'Label'}, inplace = True)
combined_df = df1.append(df2)

In [10]:
df1 = df1.reindex(columns=['Path','Label'])
df1.to_csv(os.path.join(root_dir,"freefield1010_data.csv"),index=False)
df2 = df2.reindex(columns=['Path','Label'])
df2.to_csv(os.path.join(root_dir,"Warblr_data.csv"),index=False)
combined_df = combined_df.reindex(columns=['Path','Label'])
combined_df.to_csv(os.path.join(root_dir,"combined_data.csv"),index=False)

In [12]:
df1 = df1.reindex(columns=['PathL2','Label'])
df1.to_csv(os.path.join(root_dir,"freefield1010_data.csv"),index=False)
df2 = df2.reindex(columns=['PathL2','Label'])
df2.to_csv(os.path.join(root_dir,"Warblr_data_l2.csv"),index=False)
combined_df = combined_df.reindex(columns=['PathL2','Label'])
combined_df.to_csv(os.path.join(root_dir,"combined_data_l2.csv"),index=False)

# SpeechCommandsV1

```shell
python data_prep.py --root_dir "/nlsasfs/home/nltm-pilot/sandeshk/icassp/data/SpeechCommandsV1/train" --no_workers 10 --prefix audio --suffix spec --file "labels.csv"
```

In [15]:
root_dir = root_dirs['speechcommandsv1'] 

test_labels = ["yes", "no", "up", "down","left", "right", "on", "off", "stop", "go" ] #10
core_labels = test_labels + ["zero","one","two","three","four","five","six","seven","eight","nine"] # 20
auxliary_labels = ["bird","dog","happy", "wow","bed","cat","house","marvin","sheila","tree"] #10
labels = core_labels+ auxliary_labels

train_df = pd.DataFrame(columns = [ 'Path', 'Label'])
audio_root_dir = os.path.join(root_dir , "audio")

for labe_id,label in tqdm(enumerate(labels)):
    audiofiles = os.listdir(os.path.join(audio_root_dir,label))
    for file in tqdm(audiofiles ,leave=False):
        file_path = os.path.join(label,file)
        train_df=train_df.append( {'Path' : file_path, 'Label' : label} , ignore_index = True)
train_df.to_csv(os.path.join(root_dir,"labels.csv"),index=False)

30it [03:28,  6.95s/it]


In [25]:
test_df = pd.read_csv(os.path.join(root_dir,"testing_list.txt"),header=None)
test_df.columns=['Path']
test_df['Label']= test_df.apply(lambda row: str(row.Path).split('/')[0], axis = 1)